# Abstract

Query a JWST engineering database, retrieving the list of available mnemonics, and then plotting those mnemonics over a specified time period.

Some mnemonics to start with are the spacecraft orientation menmonics, which start with `SA_ZATTEST`. Other ones which usually have values are any that deal with temperature; they will have `TEMP` in the name.

## Warning
Some mnemonics have a very high cadence/points-per-second. Choosing large time ranges, such as a day, will result in long wait times as the data is downloaded. An hour or so are good starting points.

The engineering database itself is currently only available from the internal Institute network.

# Environment

In [ ]:
# Pick some suitable intial time frame.
# And, yes, its the future. It is test data!

STIME_DEF = '2019-06-03T15:00'
ETIME_DEF = '2019-06-03T16:00'

In [ ]:
from bokeh.io import (
    output_notebook, 
    push_notebook,
    show
)
from bokeh.layouts import row
from bokeh.plotting import figure, output_notebook, show

In [ ]:
from ipywidgets import interact, fixed

In [ ]:
# Specify a different engineering database other than the standard, B-string, one.
# This will use the C-string version.
# os.environ['ENG_RESTFUL_URL'] = 'http://iwjwdmscemweb.stsci.edu/JWDMSEngFqAcc/TlmMnemonicDataSrv.svc/'

In [ ]:
import numpy as np

In [ ]:
from jwst.lib.engdb_tools import ENGDB_Service

# Library

In [ ]:
def engplot(mnemonic, stime=STIME_DEF, etime=ETIME_DEF):
    """Plot a mnemonic
    
    Parameters
    ----------
    mnemonic: str
        The mnemonic to plot
        
    stime, etime: str
        The start and end times to cover, in `YYYY-MM-DDTHH:MM::SS.sss` format.
        
    Returns
    -------
    fig: bokeh.Figure
        The Bokeh Figure object to plot.
    """
    print('Retrieving mnemonic {}'.format(mnemonic))
    values = edb.get_values(mnemonic, stime, etime, include_obstime=True, zip=False)
    print('Mnemonic {} returned {} values.'.format(mnemonic, len(values.value)))
    if not len(values.value):
        print('*** Warning: No data available to plot. Plot will be empty.')
    times = np.array([o.isot for o in values.obstime], dtype=np.datetime64)
    fig = figure(x_axis_type="datetime")
    fig.circle(times, values.value)
    return fig

In [ ]:
def engplot_update(mnemonic='SA_ZATTEST1', stime=STIME_DEF, etime=ETIME_DEF):
    """Callback to update the plot with new parameters
    
    Parameters
    ----------
    mnemonic: str
        The mnemonic to plot
        
    stime, etime: str
        The start and end times to cover, in `YYYY-MM-DDTHH:MM::SS.sss` format.
        
    Globals
    -------
    r, handle
        Pointers to the Bokeh structures to update.
    """
    try:
        p = engplot(mnemonic=mnemonic, stime=stime, etime=etime)
    except Exception as e:
        print('*** Error retrieveing mnemonic {}: Plot will not be updated'.format(mnemonic))
    else:
        r.children[0] = p
        push_notebook(handle=handle)

# Main

In [ ]:
output_notebook()

## Setup access to the engineering DB

In [ ]:
edb = ENGDB_Service()

## Get the list of mnemonics

In [ ]:
mnemonics = [m['TlmMnemonic'] for m in edb.get_meta()['TlmMnemonics']]

## Initialize the first plot

In [ ]:
fig = engplot('SA_ZATTEST1')
r = row(fig)
handle = show(r, notebook_handle=True)

## Setup the callback to update the plot with a user-specified mnemonic and time range.
The plot will automatically update when any of the fields are changed.

The status of the data retrieval appears below the input form. If there is an error or no data is available, the status will indicate what the issue is.

In [ ]:
interact(
    engplot_update, 
    mnemonic=mnemonics,
    stime=STIME_DEF,
    etime=ETIME_DEF,
    __manual=True
)